In [38]:
# Import statements 
from sklearn.linear_model import LogisticRegression
import pandas as pd
from textblob import TextBlob
import string

In [31]:
# Load data
train_feats = pd.read_csv('cleaned_feats/train_cleaned_features.csv', delimiter=',', encoding="utf-8-sig")
dev_feats = pd.read_csv('cleaned_feats/dev_cleaned_features.csv', delimiter=',', encoding="utf-8-sig")
test_feats = pd.read_csv('cleaned_feats/test_cleaned_features.csv', delimiter=',', encoding="utf-8-sig")

# Clear whitespace
train_feats.columns = train_feats.columns.str.strip()

/opt/anaconda3/envs/prop-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [36]:
# View the dataframes
train_feats.head(1)
train_feats.columns

Index(['text', 'place', 'user_location', 'followers_count', 'retweet_count',
       'favorite_count', 'hashtags', 'weekday', 'month', 'day', 'time', 'year',
       'sentiment'],
      dtype='object')

In [46]:
# Feature manipulation to make them ready for the model
# Change sentiment labels to numbers!
sentiment_map = {"positive": 1, "neutral": 0, "negative": -1}

train_feats['sentiment'] = train_feats['sentiment'].apply(lambda x: sentiment_map[str(x)])

KeyError: 'sentiment'

In [25]:
print(test_feats.head(1)['text'][0])
blob = TextBlob(test_feats.head(1)['text'][0])
print(blob.tags)           # [('The', 'DT'), ('titular', 'JJ'),
                    #  ('threat', 'NN'), ('of', 'IN'), ...]

print(blob.noun_phrases)   # WordList(['titular threat', 'blob',
                    #            'ultimate movie monster',
                    #            'amoeba-like mass', ...])

for sentence in blob.sentences:
    print(sentence.sentiment.polarity)

@Jeff_Reinebold No corona out there, that’s for sure. Enjoy
[('@', 'NNS'), ('Jeff_Reinebold', 'NNP'), ('No', 'NNP'), ('corona', 'NN'), ('out', 'IN'), ('there', 'RB'), ('that', 'IN'), ('’', 'NNP'), ('s', 'NN'), ('for', 'IN'), ('sure', 'JJ'), ('Enjoy', 'NN')]
['jeff_reinebold', '’ s', 'enjoy']
0.5
0.4


In [12]:
# Instantiate model - Multinomial logistic regression
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')